# Depuración Interactiva del Conquest Generator

Este notebook usa `record_steps=True` y muestra todos los estados intermedios (conquistas y pasos lógicos)
con un slider muy ágil, gracias a caching de imágenes y `ipywidgets.interact`.


In [ ]:
# # Si necesitas instalar:
# !pip install ipywidgets pillow

import sys, os
# Ajusta si el notebook está en un subdirectorio:
# os.chdir('..')  

# Asegúrate de que Python encuentre tu módulo:
sys.path.insert(0, os.getcwd())

from conquest_generator import generate_conquest_board  # o la ruta correcta
from PIL import Image
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from functools import lru_cache


     ------------------------------------ 139.8/139.8 kB 925.5 kB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 449.9 kB/s eta 0:00:00
     ------------------------------------ 216.6/216.6 kB 778.6 kB/s eta 0:00:00


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [ ]:
# Parámetros
N = 8
debug = True

# Ejecuta la generación con grabado de pasos
sol, final_board, iterations, successful, reason, history = generate_conquest_board(
    n=N, debug=debug, record_steps=True
)

print(f"Solución oficial: {sol}")
print(f"Iteraciones: {iterations}, Conquistas: {successful}, Motivo: {reason}")
print(f"Estados grabados: {len(history)}\n")


[debug] alternate solution at it8, revert
[debug] alternate solution at it9, revert
[debug] alternate solution at it17, revert
[debug] alternate solution at it32, revert
[debug] alternate solution at it41, revert
[debug] alternate solution at it48, revert
[debug] alternate solution at it56, revert
[debug] alternate solution at it64, revert
[debug] alternate solution at it72, revert
[debug] alternate solution at it73, revert
[debug] alternate solution at it88, revert
[debug] alternate solution at it89, revert
[debug] alternate solution at it96, revert
[debug] alternate solution at it112, revert
[debug] alternate solution at it113, revert
[debug] alternate solution at it120, revert
[debug] alternate solution at it121, revert
[stop] conquest limit 128
Solución oficial: [3, 5, 1, 7, 0, 6, 2, 4]
Iteraciones: 131, Conquistas: 128, Motivo: conquest limit 128
Estados grabados: 460



In [ ]:
@lru_cache(maxsize=None)
def get_step_image(idx: int):
    """
    Devuelve (label, PIL.Image) para el paso idx, 
    y convierte la matriz RGB a imagen sólo la primera vez.
    """
    label, board_rgb = history[idx]
    arr = np.array(board_rgb, dtype=np.uint8)  # (N,N,3)
    img = Image.fromarray(arr)
    return label, img


In [ ]:
def show_step(idx: int):
    clear_output(wait=True)
    label, img = get_step_image(idx)
    display(Markdown(f"### {idx}: {label}"))
    display(img)


IntSlider(value=0, continuous_update=False, description='Paso:', max=459)

Output()

In [ ]:
# Crea el slider
slider = widgets.IntSlider(
    value=0, min=0, max=len(history)-1, step=1, description='Paso:',
    continuous_update=False
)

# Usa interact para vincular slider → show_step
widgets.interact(show_step, idx=slider)

# Muestra inicialmente el primer paso
show_step(0)
